In [1]:
#PyTorch Workflow
import torch 
from torch import nn #nn contains all of PyTorch's neural network building blocks
import matplotlib.pyplot as plt

#check PyTorch version

torch.__version__

'2.0.0'

## 1.DATA PREPARING AND LOADING
Data can be almost anything

In [4]:
#Create *known* parameters , linear regression to make straight line
weight=0.7
bias=0.3 
#create 
start=0
end = 1
step=0.02
X = torch.arange(start,end,step).unsqueeze(dim=1)
y=weight * X + bias

X[:10],y[:10],len(X),len(y)

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50)

In [7]:
#splitting data into training and test sets.
train_split = int(0.8*len(X))
X_train ,y_train = X[:train_split],y[:train_split]
X_test, y_test = X[train_split:],y[train_split:]

len(X_train),len(y_train),len(X_test),len(y_test)

(40, 40, 10, 10)

In [ ]:
#How can we visualize our data?
